In [13]:
import os
from typing import Annotated, Literal

from pydantic import BaseModel

from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, ChatContext, Depends, inject_params


In [14]:
class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


alice_account = Account(username="alice", password="password123")
bob_account = Account(username="bob", password="password456")

account_ballace_dict = {
    (alice_account.username, alice_account.password): 300,
    (bob_account.username, bob_account.password): 200,
}


In [15]:
def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


def _get_balance(account: Account):
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"


In [16]:

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
agent = ConversableAgent(
    name="agent",
    llm_config={"config_list": config_list},
)

In [19]:
def get_balance_1(
    account: Annotated[Account, Depends(bob_account)],
    chat_context: Annotated[ChatContext, "description of chat context"],
) -> str:
    print(f"Chatcontext: {chat_context.chat_messages}")
    return _get_balance(account)

In [20]:
func = inject_params(get_balance_1)

In [21]:
from functools import wraps
from typing import Any

chat_contexts = {"chat_context": ChatContext(agent=agent)}

def update_chat_context(func, agent: "ConversableAgent"):
    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        assert len(kwargs) == 0
        for value in kwargs.values():
            if isinstance(value, ChatContext):
                value.chat_messages = agent.chat_messages()

        return func(*args, **kwargs, **chat_contexts)

    return wrapper

In [22]:
import inspect

func = update_chat_context(func, agent)

inspect.signature(func)

<Signature () -> str>

In [23]:
func()

Chatcontext: defaultdict(<class 'list'>, {})


'Your balance is 200USD'

## Real chat

In [10]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
agent = ConversableAgent(
    name="agent",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

In [11]:
@user_proxy.register_for_execution()
@agent.register_for_llm(description="Get balance")
def get_balance_1(
    account: Annotated[Account, Depends(bob_account)],
    # chat_context: Annotated[ChatContext, Depends(ChatContext(agent=agent))],
    chat_context: ChatContext = ChatContext(agent=user_proxy),
) -> str:
    print(f"context: {chat_context.chat_messages}")
    print(f"Last message: {chat_context.last_message}")
    return _get_balance(account)

In [12]:
user_proxy.initiate_chat(agent, message="Get users balance, If balance is less then 500, get it again", max_turns=5)

user_proxy_1 (to agent):

Get users balance, If balance is less then 500, get it again

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user_proxy_1):

***** Suggested tool call (call_r8ftHgWq9eleuM8dcinJ8F7b): get_balance_1 *****
Arguments: 
{}
******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_balance_1...
Call ID: call_r8ftHgWq9eleuM8dcinJ8F7b
Input arguments: {}
context: defaultdict(<class 'list'>, {<autogen.agentchat.conversable_agent.ConversableAgent object at 0x148263820>: [{'content': 'Get users balance, If balance is less then 500, get it again', 'role': 'assistant', 'name': 'user_proxy_1'}, {'tool_calls': [{'id': 'call_r8ftHgWq9eleuM8dcinJ8F7b', 'function': {'arguments': '{}', 'name': 'get_balance_1'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}]})

ChatResult(chat_id=None, chat_history=[{'content': 'Get users balance, If balance is less then 500, get it again', 'role': 'assistant', 'name': 'user_proxy_1'}, {'tool_calls': [{'id': 'call_r8ftHgWq9eleuM8dcinJ8F7b', 'function': {'arguments': '{}', 'name': 'get_balance_1'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Your balance is 200USD', 'tool_responses': [{'tool_call_id': 'call_r8ftHgWq9eleuM8dcinJ8F7b', 'role': 'tool', 'content': 'Your balance is 200USD'}], 'role': 'tool', 'name': 'user_proxy_1'}, {'tool_calls': [{'id': 'call_u7gdmhB3VKU41RpHEAYFTycw', 'function': {'arguments': '{}', 'name': 'get_balance_1'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Your balance is 200USD', 'tool_responses': [{'tool_call_id': 'call_u7gdmhB3VKU41RpHEAYFTycw', 'role': 'tool', 'content': 'Your balance is 200USD'}], 'role': 'tool', 'name': 'user_proxy_1'}, {'content': 'Your balance is currently 200 USD, which is less than 500 USD. I check